# Кратко описание на задачата
> Система за препоръки на книги, като препоръките се базират на рейтинг от читателите/потребителите. На данните би могло да се приложи ре-филтрация по възраст на читателите. Основен проблем, който би следвало да се реши преди ре-филтрацията е, че около 40% от читателите не са посочили възрастта си, което води до нужда от поправки/зачистване в данните преди ре-филтрацията, което на по-късен етап установихме, че води до загуба на информация и неточности в модела. Естествено, в следната документация предлагаме пълния процес, през който преминахме в изграждането на система за препоръки на книги.
>
> Забележка: Преди да достигнем крайната реализация на задачата си преминахме през няколко различни потенциални подхода за решаване на подобна задача, но основен подход, на който се спряхме, за реализация на задачата ни е k-NN алгоритъма във варианта му на a-NN (k-NN алгоритъм с приближение). За да изведем съображенията, които ни насочиха да тръгнем в тази посока би било редно да опишем подходите и експериментите, които проведохме, заедно с техните резултати и изводи, както и да покажем с какво a-NN подхода превъзхожда всички останали, изпробвани алгоритми в условностите на задачата ни. Повече информация за процеса на имплементация, както и за преценката за нужния ни алгоритъм може да намерите в графа Използвани алгоритми на документацията.
>
> **Забележка**: Кодът е разработен на Python със съответно необходимите му библиотеки като:
>
> 1. Numpy
> 2. pandas
> 3. sklearn.neighbors
> 4. scikit-learn (от sklearn.neighbors)
> 5. sklearn.metrics
> 6. sklearn.model_selection
> 7. scipy.stats
> 8. scipy.sparse
> 9. warnings

# Използвани данни
> Данните, които разглеждаме като CSV файлове са предоставени от **Amazon Web Services**.

## Users
Информацията, която се съхранява за читателите е техните уникални идентификационни номера (User-ID), локациите им (Location) и възрастта им.

## Books
Информацията, която се съхранява за книгите включва ISBN номерата им, загавие (Book-Title), автор (Book-Author), година на публикаци (Year-Of-Publication) и издателство на книгата (Publisher).

## Ratings
Информацията, която се съхранява за оценките на книгите включва кой потребител (User-ID) коя книга (ISBN) а е оценил и каква е била самата оценка (число в интервала от 1 до 10, вписано в полето Book-Rating).

# Организация на файловете
- docs - папка, която съдържа документацията на проекта (, тоест текущо разглеждания от Вас файл), както и папка resources и прилежащите й файлове, необходии за изграждането на документацията на проекта.
    - resources - папка, съдържаща всички необходими ресурсни файлове за създаването на документацията на проекта (например скрийншотите, включени в нея).
- public- папка, съдържаща глобално достъпни (и от двамата разработчици) файлове, необходими за осмислянето на задачата и построението на решението.
- src - папка, съдържаща данните за обработка, изходния код на системата и Jupyter notebook-а, описващ достатъчно подробно кода и прилежащите му примери. Цялото това съдържание условно се разделя в следните три папки:
    - code - папка, която съдържа изходния код на всички проведени експерименти, както и изходния код на крайния алгоритъм, използван за решаването на проблема.
    - data - папка, съдържаща всички CSV таблици, използвани в процеса на разработка на системата.
    - jupyter - папка, съдържаща ipynb файла на въпросния Jupyter notebook, използван за демонстрация на работата на системата и документация на самата демонстрация.

# Използвани алгоритми
## Експерименти

### Експеримент k-NN алгоритъм 

Импортиране на използваните dataset-ове.


In [2]:
import pandas as pd
import numpy as np
from pprint import pprint as pp
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

df_ratings = pd.read_csv("../data/Ratings.csv", na_values=["null", "nan", ""])
df_books = pd.read_csv(
    "../data/Books.csv",
    na_values=["null", "nan", ""],
    usecols=["ISBN", "Book-Title", "Book-Author"],
)
df_users = pd.read_csv("../data/Users.csv", na_values=["null", "nan", ""])

df_books = df_books.fillna("NaN")
df_ratings = df_ratings.dropna()

#### Преброяване на броя рейтинги, които са получили книгите:

- групиране на рейтингите по ISBN;
- преброяване на рейтингите;


In [3]:
combine_book_ratings = pd.merge(df_ratings, df_books, on="ISBN")
combine_book_ratings = combine_book_ratings.drop(["Book-Author"], axis="columns")

book_rating_count = (
    combine_book_ratings.groupby(by=["ISBN"])["Book-Rating"]
    .count()
    .reset_index()
    .rename(columns={"Book-Rating": "RatingCount"})
)[["ISBN", "RatingCount"]]

#### Извличане на най-оценяваните книги:

- Извличаме стойностите на квантилите на броя рейтинги в 90% на 100%.
- За статистическа значимост избираме топ 10% най-оценявани книги.


In [4]:
book_rating_with_total_count = combine_book_ratings.merge(
    book_rating_count, on=["ISBN"], how="left"
)
pp(book_rating_with_total_count["RatingCount"].quantile(np.arange(0.9, 1, 0.01)))
# Top 10% of rating counts
popularity_threshold = 136

rating_popular_books = book_rating_with_total_count.query(
    "RatingCount >= @popularity_threshold"
)

0.90    136.0
0.91    150.0
0.92    167.0
0.93    184.0
0.94    209.0
0.95    236.0
0.96    277.0
0.97    350.0
0.98    420.0
0.99    568.0
Name: RatingCount, dtype: float64


#### Дефиниране на k-NN модела:

- Използваме пивот таблиза за дефиниране на модела
  - Индекс: ISBN
  - Колони: User-ID
  - Стойности: Book-Rating
- k-NN модел
  - метрика: cosine
  - алгоритъм: auto


In [5]:
pivot = (
    rating_popular_books.drop_duplicates(["Book-Title", "User-ID"])
    .pivot(index="ISBN", columns="User-ID", values="Book-Rating")
    .fillna(0)
)

model_knn = NearestNeighbors(metric="cosine", algorithm="auto")
model_knn.fit(csr_matrix(pivot.values))

NearestNeighbors(metric='cosine')

#### Функция за прпоръки на книги:

- Използваме ISBN като индексация на пивот матрицата (таблицата).
- Използваме модела k-NN от `sklearn.neighbors`.
- След това групираме ISBN-ите на съседите и разстоянията от тях до търсената книга. 
- При грешка връщаме съобщение, че дадената книга не е сред топ 10% на оценяваните книги.


In [6]:
def get_recommends(isbn="", k_neighbors=5):
    try:
        x = pivot.loc[isbn].array.reshape(1, -1)
        distances, indices = model_knn.kneighbors(x, n_neighbors=k_neighbors)
        R_books = []
        for distance, indice in zip(distances[0], indices[0]):
            if distance != 0:
                R_book = combine_book_ratings[
                    combine_book_ratings["ISBN"] == pivot.index[indice]
                ]["Book-Title"].values[0]
                R_books.append([R_book, distance])
        recommended_books = [isbn, R_books[::-1]]
        return recommended_books
    except:
        return f"{isbn} is not in the top books"

#### Тест на системата за препоръки на книги

    Използвайки популярни ISBN-и на дадени книги, ние можем да открием най-сходните с тях.


In [7]:
pp(get_recommends("1558745157"))

['1558745157',
 [["Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1)",
   0.9381235623430844],
  ['Night Sins', 0.9352435834429496],
  ['On the Street Where You Live', 0.9258003569774721],
  ['A Child Called \\It\\": One Child\'s Courage to Survive"',
   0.7356135842239919],
  ["The Lost Boy: A Foster Child's Search for the Love of a Family",
   3.3306690738754696e-16]]]


### Експеримент k-NN алгоритъм с регресорна препоръка на книги и грануларно изчисление на дистанциите (между текущо изследвания пример и неговите k на брой най-близки съседи)

#### Обединяваме данните, които ще са ни неоходими:
- всички данни от таблицата с рейтингите и годините на читателите.
    - Годините на читателите са ни е неоходими, тъй като искаме да направим **по-комплексна препоръка** на база **рейтингите на книгите и възрастите на читателите**.
- проверяваме колко от читателите не са си споделили възрастта 
    - според статистика на сайта, от който взехме данните, такива читатели са около 40% от всички.

In [8]:
from scipy.stats import pearsonr

merged_data = pd.merge(
    df_ratings.dropna(), df_users[["User-ID", "Age"]], on="User-ID", how="inner"
)

print(f"Merged data:\n{merged_data}\n")
print(f"Merged data info:\n{merged_data.info()}\n")

Merged data:
         User-ID         ISBN  Book-Rating   Age
0         276725   034545104X            0   NaN
1         276726   0155061224            5   NaN
2         276727   0446520802            0  16.0
3         276729   052165615X            3  16.0
4         276729   0521795028            6  16.0
...          ...          ...          ...   ...
1149775   276704   1563526298            9   NaN
1149776   276706   0679447156            0  18.0
1149777   276709   0515107662           10  38.0
1149778   276721   0590442449           10  14.0
1149779   276723  05162443314            8  12.0

[1149780 rows x 4 columns]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   User-ID      1149780 non-null  int64  
 1   ISBN         1149780 non-null  object 
 2   Book-Rating  1149780 non-null  int64  
 3   Age          840288 non-null   fl

#### Запълваме NaN възрастите на потребителите с невалидни такива (тоест -1).
- Проверяваме дали всички колони в **merged_data** са с еднаква дължина, защото това ще ни е неоходима стъпка за регресията

In [9]:
merged_data = merged_data.fillna(value=-1)
print(f"Cleaned of NaN values merged data info:\n{merged_data.info()}\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   User-ID      1149780 non-null  int64  
 1   ISBN         1149780 non-null  object 
 2   Book-Rating  1149780 non-null  int64  
 3   Age          1149780 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 35.1+ MB
Cleaned of NaN values merged data info:
None



#### Подготовка на векторите, необходими за регресионния модел.
- Правим корелация по метода на Пиърсън, за да проврим съществува ли линейна връзка между рейтингите на книгите и възрастовите групи, очертали се сред нашите читатели.
    - Установяваме, че липсва линейна връзка между тези два показателя , тъй като p-value-то е равно на 0.0, тоест регресионния модел не би бил удачен избор за продължение.
- В кода си сме доразвили този процес като чист експеримент, който доведе до много лоши препоръки в потвържение на резултатите, до които доведе и корелацията. За повече детайли за този експеримент, може да разгледате кода в knn_with_regression_and_granular_computation.py в директорията src/code, както и документацията на проекта за разяснение на кода.

In [10]:
x_ratings = np.array(merged_data["Book-Rating"])
y_ages = np.array(merged_data["Age"])

correlation, p_value = pearsonr(x_ratings, y_ages)

print(f"Pearson Correlation: {correlation}")
print(
    f"P-value: {p_value}"
)  

Pearson Correlation: -0.03742512405050028
P-value: 0.0


### Експеримент a-NN
#### a-NN: HNSW разновидност

- **Импортиране на библиотеки**: Импортират се coo_matrix от scipy.sparse и библиотеката hnswlib за работа с високоефективни пространствени индекси.
- **Предобработка на данни**:
- Запълват се липсващи стойности в df_books с текст "NaN".
- Премахват се редове с липсващи стойности от df_ratings.
- Запълват се липсващи стойности в df_users с -1.
- **Обединяване на данни**: Данните за оценки на книги (df_ratings) и книги (df_books) се обединяват чрез колоната ISBN.
- **Премахване на ненужни колони**: Колоната Book-Author се премахва от резултантния DataFrame.

In [11]:
from scipy.sparse import coo_matrix
import hnswlib  # Library for HNSW

# Data preprocessing
df_books = df_books.fillna("NaN")
df_ratings = df_ratings.dropna()
df_users = df_users.fillna(-1)

combine_book_ratings = pd.merge(df_ratings, df_books, on="ISBN")
combine_book_ratings = combine_book_ratings.drop(["Book-Author"], axis="columns")

- **Групиране по ISBN**: Данните в combine_book_ratings се групират по колоната ISBN, която идентифицира книгите.
- **Изчисляване на брой оценки**: За всяка група (книга) се изчислява броят на стойностите в колоната Book-Rating с помощта на .count().
- **Презаписване на индекса**: След това се възстановява индексът с .reset_index(), за да се получи нов DataFrame.
- **Преименуване на колоната**: Колоната Book-Rating се преименува на RatingCount, за да отразява броя на оценките за всяка книга.

In [12]:
# Calculate book rating counts
book_rating_count = (
    combine_book_ratings.groupby("ISBN")["Book-Rating"]
    .count()
    .reset_index()
    .rename(columns={"Book-Rating": "RatingCount"})
)

- **Обединяване на данни**: Данните от combine_book_ratings се обединяват с book_rating_count чрез колоната ISBN, която е общата за двата DataFrame-а.
- **Метод на обединяване**: Използва се методът left join (how="left"), което означава, че всички редове от combine_book_ratings ще бъдат запазени, а стойностите от book_rating_count ще се добавят, ако има съвпадение по ISBN.
- **Резултат**: В резултат ще се получи нов DataFrame book_rating_with_total_count, който съдържа оригиналните данни от combine_book_ratings, но с добавена информация за броя на оценките за всяка книга.

In [13]:
# Merge rating counts with the ratings data
book_rating_with_total_count = combine_book_ratings.merge(
    book_rating_count, on="ISBN", how="left"
)

- **Дефиниране на праг за популярност**: Задава се прагово значение за броя на оценките на книгите (popularity_threshold = 136).
- **Филтриране на книги по популярност**: Използва се метода .query() за филтриране на редовете от book_rating_with_total_count, при които стойността на RatingCount е по-голяма или равна на зададения праг.
- **Копиране на резултата**: Резултатът от филтрирането се копира в нов DataFrame rating_popular_books, който съдържа само тези книги, които имат равен или по-голям брой оценки от зададения праг.

In [14]:
# Filter by popularity threshold
popularity_threshold = 136
rating_popular_books = book_rating_with_total_count.query(
    "RatingCount >= @popularity_threshold"
).copy()

- **Групиране по потребителски идентификатор**: Данните в rating_popular_books се групират по колоната User-ID (потребителски идентификатор), и за всяка група се изчислява броят на оценките на книги, които потребителят е оставил, чрез .count().
- **Филтриране на активни потребители**: Потребители, които имат по-малко от 5 оценки, се считат за неактивни. Активните потребители са тези, които са оставили поне 5 оценки. Създава се индексиран списък active_users, който съдържа идентификаторите на тези потребители.
- **Филтриране на данни по активни потребители**: Останалите редове в rating_popular_books, които съдържат оценки от неактивни потребители (с по-малко от 5 оценки), се премахват. Остават само тези оценки, които са направени от активни потребители, чрез използване на .isin(active_users).

In [15]:
# Filter out inactive users
user_activity = rating_popular_books.groupby("User-ID")["Book-Rating"].count()
active_users = user_activity[user_activity >= 5].index
rating_popular_books = rating_popular_books[
    rating_popular_books["User-ID"].isin(active_users)
]

- **Създаване на mapping за потребителите**:
    - Създава се речник user_id_mapping, който свързва уникалните потребителски идентификатори (от колоната User-ID в rating_popular_books) с уникални индекси. Използва се enumerate() за да се генерира индекс за всеки уникален потребител.
- **Създаване на mapping за книгите**:
    - Създава се речник isbn_mapping, който свързва уникалните идентификатори на книги (от колоната ISBN) с уникални индекси, като отново се използва enumerate().

In [16]:
# Precompute mappings for users and books
user_id_mapping = {
    user_id: idx for idx, user_id in enumerate(rating_popular_books["User-ID"].unique())
}
isbn_mapping = {
    isbn: idx for idx, isbn in enumerate(rating_popular_books["ISBN"].unique())
}

- **Мапиране на потребителски идентификатори към индекси**:
    - Колоната User-ID в DataFrame rating_popular_books се заменя с индекси от user_id_mapping чрез метода .map(). Резултатът се съхранява в новата колона User-Idx.
- **Мапиране на идентификатори на книги към индекси**:
    - Колоната ISBN в rating_popular_books също се заменя с индекси от isbn_mapping чрез метода .map(). Резултатът се записва в новата колона ISBN-Idx.

In [17]:
# Map User-ID and ISBN to respective indices
rating_popular_books["User-Idx"] = rating_popular_books["User-ID"].map(user_id_mapping)
rating_popular_books["ISBN-Idx"] = rating_popular_books["ISBN"].map(isbn_mapping)

- **Създаване на разреден матрица (sparse matrix)**:
    - Използва се функцията coo_matrix от библиотеката scipy.sparse, за да се създаде разредена матрица. В този случай, разредената матрица ще съдържа оценките на книги (Book-Rating).
    - Индексите на редовете на матрицата са стойностите в ISBN-Idx (индексираните идентификатори на книгите).
    - Индексите на колоните на матрицата са стойностите в User-Idx (индексираните идентификатори на потребителите).
    - Размерността на матрицата е определена чрез броя на уникалните книги (използвайки len(isbn_mapping)) и броя на уникалните потребители (използвайки len(user_id_mapping)).
- **Конвертиране на матрицата в CSR формат**:
    - След създаването на разредената матрица в COO формат (която е подходяща за ефективно добавяне на нови стойности), тя се конвертира в CSR (Compressed Sparse Row) формат чрез метода .tocsr(). CSR е по-ефективен за операции като умножение на матрици и е подходящ за алгоритми за препоръки.

In [18]:
# Create sparse matrix
sparse_matrix = coo_matrix(
    (
        rating_popular_books["Book-Rating"],
        (rating_popular_books["ISBN-Idx"], rating_popular_books["User-Idx"]),
    ),
    shape=(len(isbn_mapping), len(user_id_mapping)),
).tocsr()

- **Обратен мапинг за книгите**:
    - Създава се речник index_to_isbn, който свързва индекси (стойности от isbn_mapping) с техните съответстващи ISBN стойности. Това е обратният процес на мапинга на ISBN към индексите, който е бил извършен по-рано.
- **Обратен мапинг за потребителите**:
    - Създава се речник index_to_user_id, който свързва индекси (стойности от user_id_mapping) с техните съответстващи User-ID стойности. Това е обратният процес на мапинга на User-ID към индексите, извършен по-рано.

In [19]:
# Reverse mappings for later use
index_to_isbn = {idx: isbn for isbn, idx in isbn_mapping.items()}
index_to_user_id = {idx: user_id for user_id, idx in user_id_mapping.items()}

- Определяне на размерността:
    - dim = sparse_matrix.shape[1] извлича броя на колоните в разредената матрица sparse_matrix, което представлява броя на потребителите (т.е. характеристиките или размерността на данните за всеки елемент). Това е използвано за дефиниране на размерността на индекса в HNSW.
- Инициализация на HNSW индекс:
    - Създава се HNSW индекс чрез библиотеката hnswlib, като се използва косинусово разстояние (space="cosine") за измерване на сходството между елементите. Това означава, че индексът ще търси най-близките съседи чрез косинусова метрика.
    - Параметърът dim=dim задава размерността на пространството (в този случай броя на потребителите).
- Инициализация на индекс с параметри:
    - Извиква се index.init_index(max_elements=sparse_matrix.shape[0], ef_construction=200, M=16) за инициализиране на HNSW индекса:
    - max_elements=sparse_matrix.shape[0] задава максималния брой елементи, които индексът ще съдържа (тук броят на уникалните книги).
    - ef_construction=200 задава параметър за контрол на качеството на индекса по време на изграждането му (по-висока стойност осигурява по-добро качество, но с по-високи изисквания за време).
    - M=16 задава броя на връзките, които всяка точка ще има в индекса, което влияе на баланса между производителност и точност.

In [20]:
# Initialize HNSW index using hnswlib
dim = sparse_matrix.shape[1]  # Number of features (users)
index = hnswlib.Index(space="cosine", dim=dim)  # Use cosine distance for similarity
index.init_index(max_elements=sparse_matrix.shape[0], ef_construction=200, M=16)

- Добавяне на данни в индекса:
    - index.add_items(sparse_matrix.toarray()) добавя данни в HNSW индекса. Разредената матрица sparse_matrix се преобразува в обикновена (плътна) матрица чрез .toarray() и се добавя към индекса. Това позволява на индекса да съхранява оценките на книгите и потребителите като векторни представяния.
- Настройване на параметър за търсене (efSearch):
    - index.set_ef(50) настройва параметъра efSearch, който контролира точността и бързината на търсенето в индекса. По-високото значение на efSearch прави търсенето по-точно, но и по-бавно, тъй като индексът ще разглежда повече възможни съседи при търсене на най-близки елементи.

In [21]:
# Add data to the index
index.add_items(sparse_matrix.toarray())
# Set efSearch for querying (larger value makes the search more accurate but slower)
index.set_ef(50)

- Групиране на данни по ISBN: Данните в rating_popular_books се групират по колоната ISBN (идентификатор на книга), като за всяка група се изчислява размерът ѝ с .size(). Това дава броя на оценките за всяка книга.
- Сортиране на книгите по брой оценки: Сортират се резултатите по броя на оценките (в низходящ ред) с помощта на .sort_values(ascending=False).
- Избор на най-популярните 10 книги: Избира се първите 10 книги (най-популярните) от сортирания резултат чрез .index[:10], като се извличат само идентификаторите на тези книги и се преобразуват в списък с .tolist().

In [22]:
# Fallback to k most popular books
most_popular_books = (
    rating_popular_books.groupby("ISBN")
    .size()
    .sort_values(ascending=False)
    .index[:10]
    .tolist()
)

- Дефиниция на функцията:
    - get_most_popular(isbn, k_neighbors=5) приема два аргумента:
        - isbn: Идентификатор на книгата, за която искаме да намерим препоръки (това може да бъде използвано за логика на препоръки или като част от по-голям контекст).
        - k_neighbors=5: Броят на най-популярните книги, които да се върнат като препоръки (по подразбиране е 5).
- Резервни препоръки:
    - Създава се списък recommended_books, който съдържа препоръчани книги от най-популярните книги (първите k_neighbors книги от списъка most_popular_books).
    - За всяка книга от списъка most_popular_books[:k_neighbors], се извлича заглавието на книгата от combine_book_ratings, като се използва ISBN за намиране на съответната книга.
    - Списъкът за всяка книга съдържа две стойности:
        - Заглавието на книгата и самия ISBN.
        - Стойност 0, което вероятно показва, че тези препоръки са базирани на популярност и нямат оценка (или са с минимална стойност).
- Връщане на резултат:
    - Функцията връща списък с два елемента:
        - isbn: първоначалния идентификатор на книгата.
        - recommended_books: списък с препоръчани книги, базирани на популярност.

In [23]:
def get_most_popular(isbn, k_neighbors=5):
    # Fallback to most popular books
    recommended_books = [
        [
            [
                combine_book_ratings[combine_book_ratings["ISBN"] == book][
                    "Book-Title"
                ].values[0],
                book,
            ],
            0,
        ]
        for book in most_popular_books[:k_neighbors]
    ]
    return [isbn, recommended_books]

- Валидация на ISBN: Проверява дали подаденият isbn е валиден, като се търси в мапинга isbn_mapping.
- Извличане на вектор за търсене: Ако ISBN е валиден, намира индексa му в матрицата и извлича съответния вектор (представяне на книгата) от разредената матрица sparse_matrix.
- Търсене на най-близки съседи: Използва метода knn_query на HNSW индекса, за да намери k_neighbors + 1 най-близки съседи на подадената книга, използвайки разстояния и индекси.
- Генериране на препоръки: Препоръчва книги, като се филтрират резултатите, за да се премахнат книги, които имат разстояние >= 1 или са същата книга като търсената. Препоръките се връщат заедно с разстоянията.
- Резервни препоръки: Ако възникне грешка или ISBN не е валиден, се извиква функцията get_most_popular, която предлага най-популярните книги.

In [24]:
# Recommendation function
def get_recommends(isbn, k_neighbors=5):
    try:
        # Validate ISBN
        if isbn in isbn_mapping:
            # Get query vector
            isbn_idx = isbn_mapping[isbn]
            query_vector = sparse_matrix[isbn_idx].toarray().flatten()
            # Query the HNSW index
            indices, distances = index.knn_query(query_vector, k=k_neighbors + 1)
            # Generate recommendations
            recommendations = []
            for distance, indice in zip(distances[0], indices[0]):
                recommended_isbn = index_to_isbn[indice]
                if (
                    distance < 1 and recommended_isbn != isbn
                ):  # Filter for meaningful results
                    book_title_isbn = get_title_isbn(recommended_isbn)
                    recommendations.append([book_title_isbn, distance])
            return [isbn, recommendations[::-1]]
        else:
            return get_most_popular(isbn, k_neighbors)
    except Exception as e:
        pp("ERROR")
        pp(str(e.with_traceback()))
        return get_most_popular(isbn, k_neighbors)

- Дефиниция на функцията:
    - Функцията приема един аргумент: isbn, който е идентификатор на книга (ISBN номер).
- Извличане на заглавие на книга:
    - combine_book_ratings[combine_book_ratings["ISBN"] == isbn]: Тази част от кода търси всички редове в DataFrame combine_book_ratings, където колоната ISBN съвпада с подадения isbn.
    - values[0]: След като е намерен редът или редовете с даденото ISBN, .values[0] извлича първото (и най-вероятно единствено) заглавие на книга от колоната Book-Title.
- Връщане на резултата:
    - Функцията връща списък, съдържащ два елемента:
        - Заглавието на книгата (от колоната Book-Title).
        - Самия ISBN, който беше подаден като аргумент.

In [25]:
def get_title_isbn(isbn):
    return [
        combine_book_ratings[combine_book_ratings["ISBN"] == isbn]["Book-Title"].values[
            0
        ],
        isbn,
    ]

- Тестване на препоръки:

    - Създава се променлива example_isbn, която съдържа примерен ISBN ("1558745157"). Това е ISBN на книга, за която ще се извлекат препоръки.
    - Функцията get_recommends(example_isbn, k_neighbors=5) се извиква, за да се получат препоръки за книгата с този ISBN. k_neighbors=5 указва, че искаме да получим 5 препоръчани книги.
    - Резултатът от препоръките се извежда с помощта на pp() (което вероятно е съкращение за pprint, използвано за по-четливото отпечатване на сложни структури от данни).
- Тестване на друга книга:
    - Извиква се същата функция get_recommends и за друга книга, с ISBN "0330281747", като отново се изискват 5 препоръки.

In [26]:
# Test example
example_isbn = "1558745157"  # Replace with an ISBN from your dataset
pp(get_recommends(example_isbn, k_neighbors=5))
pp(get_recommends("0330281747"))

['1558745157',
 [[['Night Sins', '055356451X'], 0.9014358],
  [['Icy Sparks', '0142000205'], 0.90126425],
  [["Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1)",
    '0842329129'],
   0.89481175],
  [['On the Street Where You Live', '0671004530'], 0.8627482],
  [['A Child Called \\It\\": One Child\'s Courage to Survive"', '1558743669'],
   0.63628924]]]
['0330281747',
 [[['Wild Animus', '0971880107'], 0],
  [['The Lovely Bones: A Novel', '0316666343'], 0],
  [['The Da Vinci Code', '0385504209'], 0],
  [['Divine Secrets of the Ya-Ya Sisterhood: A Novel', '0060928336'], 0],
  [['The Red Tent (Bestselling Backlist)', '0312195516'], 0]]]


#### a-NN: FAISS разновидност

- Импортиране на библиотеки:
    - Импортира се faiss, библиотека за ефективно търсене на най-близки съседи.
    - Импортира се normalize от sklearn.preprocessing, който ще бъде използван за нормализиране на данни.
    - Импортира се TfidfVectorizer от sklearn.feature_extraction.text, който ще се използва за извличане на TF-IDF характеристики от текстови данни.
- Обединяване на данни:
    - Двата DataFrame df_ratings (с оценки на книги) и df_books (с метаданни на книги) се обединяват чрез ISBN с pd.merge.
    - След обединяването се премахва колоната Book-Author, тъй като тя не се използва в последващия анализ.

In [27]:
import faiss
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer

# Merge ratings with book metadata
combine_book_ratings = pd.merge(df_ratings, df_books, on="ISBN")
combine_book_ratings = combine_book_ratings.drop(["Book-Author"], axis="columns")

- Изчисляване на броя на оценките за всяка книга:
    - combine_book_ratings.groupby(by=["ISBN"])["Book-Rating"].count(): Групира данните по ISBN и брои колко оценки има всяка книга.
    - Резултатите се преобразуват в DataFrame, като колоната с броя на оценките се преименува на RatingCount.
- Добавяне на популярността към данните:
    - Данните с броя на оценките (book_rating_count) се обединяват с основния DataFrame combine_book_ratings, използвайки ISBN като ключ.
    - Полученият DataFrame, book_rating_with_total_count, съдържа информация за всяка оценка, заедно с общия брой оценки за съответната книга.

In [28]:
# Calculate popularity threshold
book_rating_count = (
    combine_book_ratings.groupby(by=["ISBN"])["Book-Rating"]
    .count()
    .reset_index()
    .rename(columns={"Book-Rating": "RatingCount"})
)[["ISBN", "RatingCount"]]

book_rating_with_total_count = combine_book_ratings.merge(
    book_rating_count, on=["ISBN"], how="left"
)

- Квантили на броя на оценките:
    - book_rating_with_total_count["RatingCount"].quantile(np.arange(0.9, 1, 0.01)): Изчислява квантили между 90% и 100% (с 1% стъпка) за колоната - RatingCount. Това показва как броят на оценките е разпределен сред книгите в горния диапазон на популярност.
- Дефиниране на прагове за популярност:
    - popularity_threshold = 136: Определя се прагът за популярност — книги с 136 или повече оценки ще се считат за популярни.
- Филтриране на популярни книги:
    - popular_books: Съдържа само тези книги, които имат RatingCount >= popularity_threshold.
- Филтриране на непопулярни книги:
    - unpopular_books: Включва книги с броя на оценките, който е:
        - По-малък от popularity_threshold.
        - По-голям от 0 (за да изключи книги без никакви оценки).

In [29]:
# Thresholds
pp(book_rating_with_total_count["RatingCount"].quantile(np.arange(0.9, 1, 0.01)))
popularity_threshold = 136
popular_books = book_rating_with_total_count.query(
    "RatingCount >= @popularity_threshold"
)
unpopular_books = book_rating_with_total_count.query(
    "RatingCount < @popularity_threshold & RatingCount > 0"
)

0.90    136.0
0.91    150.0
0.92    167.0
0.93    184.0
0.94    209.0
0.95    236.0
0.96    277.0
0.97    350.0
0.98    420.0
0.99    568.0
Name: RatingCount, dtype: float64


- Премахване на дублиращи се записи:
    - popular_books.drop_duplicates(["ISBN", "User-ID"]): Премахват се дублиращи се комбинации от ISBN (идентификатор на книга) и User-ID (идентификатор на потребител), за да се гарантира уникалност.
- Създаване на пивотна таблица:
    - .pivot(index="ISBN", columns="User-ID", values="Book-Rating"): Трансформира данните в пивотна таблица, където:
        - Редовете (index) са ISBN на книгите.
        - Колоните (columns) са User-ID на потребителите.
        - Стойностите (values) са оценките на книгите (Book-Rating).
- Попълване на липсващи стойности:
    - .fillna(0): Замества липсващите стойности в матрицата с 0, за да се улесни обработката.

In [30]:
# Create embeddings for popular books
pivot_popular = (
    popular_books.drop_duplicates(["ISBN", "User-ID"])
    .pivot(index="ISBN", columns="User-ID", values="Book-Rating")
    .fillna(0)
)

- Нормализиране на векторите:
    - normalize(pivot_popular.values, axis=1, norm="l2"):
        - Нормализира редовете (вектори на книги) в pivot_popular.values, използвайки L2-норма.
        - Това преобразува всеки вектор така, че дължината му да стане 1, което улеснява изчисляването на косинусова сходност.
    - Създаване на списък с ISBN номера:
        - isbn_list_popular = pivot_popular.index.tolist():
            - Взема индекса на pivot_popular (ISBN номера на книгите) и го преобразува в списък.

In [31]:
# Normalize vectors for cosine similarity
popular_embeddings = normalize(pivot_popular.values, axis=1, norm="l2")
isbn_list_popular = pivot_popular.index.tolist()

- Извличане на размерността:
    - d = popular_embeddings.shape[1]: Взема размерността на вгражданията (брой характеристики), която е броят на колоните в матрицата  popular_embeddings.
- Създаване на Faiss индекс:
    - faiss.IndexFlatIP(d):
        - Създава индекс за търсене с вътрешен произведен (inner product), което е еквивалентно на косинусова сходност, когато векторите са  нормализирани (както е направено с L2-нормализация по-рано).
- Добавяне на вграждания към индекса:
    - index.add(popular_embeddings):
        - Добавя нормализираните вектори (popular_embeddings) към създадения Faiss индекс, за да бъдат готови за търсене.

In [32]:
# Index popular book embeddings with Faiss
d = popular_embeddings.shape[1]
index = faiss.IndexFlatIP(d)  # Cosine similarity
index.add(popular_embeddings)

- Подготовка на данните
    - Извличане на непопулярни книги:
        - unpopular_titles: Съдържа записи от df_books, които съответстват на непопулярни книги (ISBN-та от unpopular_books).
    - Обработка на празни заглавия:
        - Заменя NaN стойности в колоната Book-Title с "Unknown Title".
- TF-IDF векторизация
    - Създаване на TF-IDF вектори:
        - TfidfVectorizer(max_features=2000, lowercase=True): Векторизира заглавията на книги, използвайки максимум 2000 характеристики и преобразува текста в малки букви.
        - vectorizer.fit_transform(unpopular_titles["Book-Title"]).toarray(): Генерира TF-IDF матрица за заглавията на книгите.
    - Нормализация на векторите:
        - Нормализира векторите с L2-норма, за да улесни изчисляването на косинусова сходност.
- Проверка за празни вектори
    - Филтриране на празни вектори:
        - Векторите със сумарна стойност от 0 (празни или без смислена информация) се премахват.
        - valid_rows идентифицира валидните редове, а съответстващите ISBN се филтрират.
- Създаване на Faiss индекс
    - Faiss индекс за непопулярни книги:
        - index_unpopular = faiss.IndexFlatIP(tfidf_embeddings.shape[1]): Създава индекс за косинусова сходност.
        - index_unpopular.add(tfidf_embeddings): Добавя TF-IDF вектори за заглавията към индекса.
- Резултат
    - tfidf_embeddings: Нормализирани TF-IDF вектори за валидни заглавия на непопулярни книги.
    - isbn_list_unpopular: Списък с ISBN номера на тези книги.
    - index_unpopular: Faiss индекс, който позволява търсене на подобни книги по заглавие за непопулярни книги.

In [33]:
# Handle unpopular books using TF-IDF on book titles
vectorizer = TfidfVectorizer(max_features=2000, lowercase=True)
unpopular_titles = df_books[df_books["ISBN"].isin(unpopular_books["ISBN"])]
unpopular_titles.loc[unpopular_titles["Book-Title"].isna(), "Book-Title"] = (
    "Unknown Title"
)

tfidf_embeddings = vectorizer.fit_transform(unpopular_titles["Book-Title"]).toarray()
isbn_list_unpopular = unpopular_titles["ISBN"].tolist()
if tfidf_embeddings.shape[0] > 0:
    tfidf_embeddings = normalize(tfidf_embeddings, axis=1, norm="l2")

pp(f"TF-IDF embeddings shape: {tfidf_embeddings.shape}")
pp(f"Zero embeddings count: {(tfidf_embeddings.sum(axis=1) == 0).sum()}")

valid_rows = tfidf_embeddings.sum(axis=1) > 0
tfidf_embeddings = tfidf_embeddings[valid_rows]
isbn_list_unpopular = [
    isbn for valid, isbn in zip(valid_rows, isbn_list_unpopular) if valid
]

pp(f"TF-IDF embeddings shape: {tfidf_embeddings.shape}")
pp(f"Zero embeddings count: {(tfidf_embeddings.sum(axis=1) == 0).sum()}")

index_unpopular = faiss.IndexFlatIP(tfidf_embeddings.shape[1])
index_unpopular.add(tfidf_embeddings)

C:\Users\Sybatron\AppData\Local\Temp\ipykernel_38676\613658435.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpopular_titles.loc[unpopular_titles["Book-Title"].isna(), "Book-Title"] = (


'TF-IDF embeddings shape: (269717, 2000)'
'Zero embeddings count: 19920'
'TF-IDF embeddings shape: (249797, 2000)'
'Zero embeddings count: 0'


- Групиране по ISBN:
    - popular_books.groupby("ISBN").size(): Групира данните за популярни книги по ISBN и изчислява броя на записите за всяка книга, което съответства на броя на потребителите, които са я оценили.
- Сортиране по популярност:
    - .sort_values(ascending=False): Сортира книгите в низходящ ред по броя на оценките (най-популярните първи).
- Избиране на топ 10 най-популярни книги:
    - .index[:10].tolist(): Взема първите 10 ISBN номера от сортирания списък и ги преобразува в списък.

In [34]:
# Fallback to k most popular books
most_popular_books = (
    popular_books.groupby("ISBN")
    .size()
    .sort_values(ascending=False)
    .index[:10]
    .tolist()
)

- Параметри:
    - isbn: ISBN номерът на книгата, за която се иска препоръка.
    - k_neighbors=5: Броят книги за препоръка (по подразбиране 5).
- Създаване на препоръки:
    - Цикъл през най-популярните книги:
        - for book in most_popular_books[:k_neighbors]: Избира първите k_neighbors книги от списъка с най-популярни книги.
- Извличане на заглавие и ISBN:
    - combine_book_ratings[combine_book_ratings["ISBN"] == book]["Book-Title"].values[0]: Намира заглавието на книгата от combine_book_ratings по нейния ISBN.
- Формат на препоръката:
    - Всяка препоръка е представена като:
        [[<Book-Title>, <ISBN>], 0]
        - <Book-Title>: Заглавието на книгата.
        - <ISBN>: ISBN номерът на книгата.
        - 0: Разстоянието или сходството (фиксирано на 0 в този случай, тъй като препоръките не се базират на сходство).
- Връщане на резултата:
    - Списък с препоръки: return [isbn, recommended_books]: Връща текущия isbn и списъка с препоръки.

In [35]:
def get_most_popular(isbn, k_neighbors=5):
    # Fallback to most popular books
    recommended_books = [
        [
            [
                combine_book_ratings[combine_book_ratings["ISBN"] == book][
                    "Book-Title"
                ].values[0],
                book,
            ],
            0,
        ]
        for book in most_popular_books[:k_neighbors]
    ]
    return [isbn, recommended_books]

- Параметри:
    - isbn: ISBN номерът на книгата, за която се търсят препоръки.
    - k_neighbors=5: Броят препоръчани книги.
- Обработка на популярни книги:
    - Проверява дали isbn е в списъка с популярни книги (isbn_list_popular).
    - Намира най-близките съседи, използвайки Faiss индекс върху вгражданията (popular_embeddings).
    - Добавя препоръки, базирани на сходство (дистанция > 0 и различен ISBN).
- Обработка на непопулярни книги:
    - Проверява дали isbn е в списъка с непопулярни книги (isbn_list_unpopular).
    - Ако има валидни TF-IDF вграждания, използва Faiss индекс върху тях.
    - Генерира препоръки, като изключва текущата книга и резултати с нулева дистанция.
- Резервна стратегия:
    - Ако isbn не е намерен в нито един от списъците или има грешка, връща най-популярните книги чрез get_most_popular.
- Резултат:
    - Връща списък с препоръки във формат [isbn, recommended_books], където всяка препоръка съдържа:
        - Заглавие и ISBN на препоръчаната книга.
        - Сходство/дистанция между книгите.

In [36]:
# Recommendation function
def get_recommends(isbn, k_neighbors=5):
    try:
        if isbn in isbn_list_popular:
            idx = isbn_list_popular.index(isbn)
            distances, indices = index.search(
                popular_embeddings[idx].reshape(1, -1), k_neighbors + 1
            )
            recommended_books = [
                [get_title_isbn(isbn_list_popular[i]), distances[0][j]]
                for j, i in enumerate(indices[0])
                if distances[0][j] > 0 and isbn_list_popular[i] != isbn
            ]
        elif isbn in isbn_list_unpopular:
            if tfidf_embeddings.shape[0] <= 0:
                pp("No valid TF-IDF embeddings. Falling back to most popular books.")
                return get_most_popular(isbn, k_neighbors)
            idx = isbn_list_unpopular.index(isbn)
            distances, indices = index_unpopular.search(
                normalize(tfidf_embeddings[idx].reshape(1, -1)), k_neighbors + 1
            )
            recommended_books = [
                [get_title_isbn(isbn_list_unpopular[i]), distances[0][j]]
                for j, i in enumerate(indices[0])
                if distances[0][j] > 0 and isbn_list_unpopular[i] != isbn
            ]
        else:
            return get_most_popular(isbn, k_neighbors)
    except Exception as e:
        pp("ERROR")
        pp(str(e.with_traceback()))
        return get_most_popular(isbn, k_neighbors)
    return [isbn, recommended_books]

- Търсене на заглавие по ISBN:
    - combine_book_ratings[combine_book_ratings["ISBN"] == isbn]: Филтрира данни, за да намери редовете, където ISBN съвпада с подадения isbn.
    ["Book-Title"].values[0]: Извлича първото заглавие (Book-Title) от филтрираните данни.
- Резултат:
    - Връща списък с два елемента:
        - Заглавието на книгата.
        - ISBN номерът на книгата.

In [37]:
def get_title_isbn(isbn):
    return [
        combine_book_ratings[combine_book_ratings["ISBN"] == isbn]["Book-Title"].values[
            0
        ],
        isbn,
    ]


Тестваме функцията за препоръки на книги.

In [38]:
# Test the function
pp(get_recommends("1558745157"))
pp(get_recommends("0330281747"))

['1558745157',
 [[['A Child Called \\It\\": One Child\'s Courage to Survive"', '1558743669'],
   0.26438642],
  [['On the Street Where You Live', '0671004530'], 0.07419965],
  [['Night Sins', '055356451X'], 0.064756416],
  [["Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1)",
    '0842329129'],
   0.061876435],
  [['L Is for Lawless', '0449221490'], 0.061851475]]]
['0330281747',
 [[['The crystal bucket: Television criticism from the Observer, 1976-79',
    '0224018906'],
   0.7108932],
  [['The Box: An Oral History of Television, 1920-1961', '0140252657'],
   0.6335434],
  [['Channels of Discourse, Reassembled: Television and Contemporary Criticism',
    '0807843741'],
   0.61701965],
  [['The Television Detectives, 1948-1978', '0498022366'], 0.57000047],
  [['The Houdini Box', '0679814299'], 0.54777056]]]


#### a-NN: ANNOY разновидност

- Изчисляване на популярността на книгите:
    - Групиране по ISBN и броене на оценки:
        - combine_book_ratings.groupby(by=["ISBN"])["Book-Rating"].count(): Групира данните по ISBN и брои колко пъти е оценявана всяка книга.
    - Преобразуване на резултата в DataFrame:
        - .reset_index(): Съсредоточава резултата в DataFrame и връща индексите обратно към редове.
        - .rename(columns={"Book-Rating": "RatingCount"}): Преименува колоната с броя на оценките на RatingCount за по-ясно наименование.
- Обединяване на броя на оценките с основните данни:
    - Сливане на данните:
        - combine_book_ratings.merge(book_rating_count, on=["ISBN"], how="left"): Извършва ляво сливане (left join) между основния DataFrame combine_book_ratings и book_rating_count по колоната ISBN. Това добавя информация за броя на оценките за всяка книга към основния DataFrame.

In [39]:
from annoy import AnnoyIndex

# Compute book popularity
book_rating_count = (
    combine_book_ratings.groupby(by=["ISBN"])["Book-Rating"]
    .count()
    .reset_index()
    .rename(columns={"Book-Rating": "RatingCount"})
)[["ISBN", "RatingCount"]]

book_rating_with_total_count = combine_book_ratings.merge(
    book_rating_count, on=["ISBN"], how="left"
)

- Изчисляване на квантилите:
    - book_rating_with_total_count["RatingCount"].quantile(np.arange(0.9, 1, 0.01)):
        - Изчислява квантилите на броя на оценките (RatingCount) за книгите, като използва стойности в интервала от 0.9 до 1 с интервал от 0.01. Това показва разпределението на броя на оценките в този диапазон, което може да помогне за определяне на прага на популярност.
- Задаване на праг на популярност:
    - popularity_threshold = 136: Установява праг от 136 за минималния брой оценки, който една книга трябва да има, за да бъде считана за популярна.
- Филтриране на популярни книги:
    - book_rating_with_total_count.query("RatingCount >= @popularity_threshold"): Използва метод .query() за да филтрира книгите, които имат брой оценки (RatingCount) по-голям или равен на popularity_threshold (136). Това създава нов DataFrame, съдържащ само популярни книги.

In [40]:
# Thresholds
pp(book_rating_with_total_count["RatingCount"].quantile(np.arange(0.9, 1, 0.01)))
popularity_threshold = 136

popular_books = book_rating_with_total_count.query(
    "RatingCount >= @popularity_threshold"
)

0.90    136.0
0.91    150.0
0.92    167.0
0.93    184.0
0.94    209.0
0.95    236.0
0.96    277.0
0.97    350.0
0.98    420.0
0.99    568.0
Name: RatingCount, dtype: float64


- Изграждане на матрица на потребителите и книгите:
    - Преобразуване на данни в пивотирана таблица:
        - popular_books.drop_duplicates(["ISBN", "User-ID"]): Премахва дублиращи се записи за всяка комбинация от потребител и книга, така че всяка книга да има само една оценка от даден потребител.
        - .pivot(index="ISBN", columns="User-ID", values="Book-Rating"): Създава пивотирана таблица, където:
    - Индексът на редовете е ISBN (книгите).
    - Колоните са User-ID (потребителите).
    - Стойностите в клетките са оценките на книгите (колоната Book-Rating).
        - .fillna(0): Попълва липсващите стойности с 0, което означава, че ако даден потребител не е оценил дадена книга, стойността ще бъде 0.
2. Създаване на разредена матрица:
    - sparse_matrix = csr_matrix(user_book_matrix.values): Преобразува пивотирана таблица в разредена матрица (CSR формат), използвайки метод csr_matrix. Това е ефективен начин за съхранение на големи матрици, които съдържат много нули.

In [41]:
# Popular annoy

# Sparse matrix
user_book_matrix = (
    popular_books.drop_duplicates(["ISBN", "User-ID"])
    .pivot(index="ISBN", columns="User-ID", values="Book-Rating")
    .fillna(0)
)

sparse_matrix = csr_matrix(user_book_matrix.values)

- Настройване на параметрите за Annoy Index:
    - num_features = sparse_matrix.shape[1]: Това определя броя на характеристиките (потребителите) в разредената матрица, като взема броя на колоните (shape[1]), т.е. колко различни потребители има в базата данни.
    - annoy_index = AnnoyIndex(num_features, metric="angular"): Създава нов индекс Annoy с определен брой характеристики (num_features) и използва "angular" метриката за изчисляване на разстояния (ангуларно разстояние, което е подходящо за вектори, нормализирани за косинусно сходство).
- Мапинг на ISBN и индекси:
    - isbn_to_index = {isbn: idx for idx, isbn in enumerate(user_book_matrix.index)}: Създава речник, който свързва ISBN на книга с индекс в пивотирания DataFrame user_book_matrix. Това позволява бързо намиране на индекс на книга по нейния ISBN.
    - index_to_isbn = {idx: isbn for isbn, idx in isbn_to_index.items()}: Създава обратен речник, който свързва индекс с ISBN. Това е полезно за връщане на резултатите от индекса в оригиналния формат.
- Добавяне на елементи към Annoy Index:
    - for idx, row in enumerate(sparse_matrix):: Итерация през всяка ред на разредената матрица (sparse_matrix), който представлява оценките на книгата от всички потребители.
    - annoy_index.add_item(idx, row.toarray()[0]): Добавя всеки ред от разредената матрица към Annoy индекса. Всеки ред е преобразуван в масив (toarray()[0]), който съдържа оценките за съответната книга.
- Изграждане на индекса:
    - annoy_index.build(n_trees=10): Изгражда Annoy индекс с 10 дървета. Повече дървета обикновено водят до по-точни резултати, но и по-бавно изграждане.

In [42]:
# Build Annoy Index for all books
num_features = sparse_matrix.shape[1]
annoy_index = AnnoyIndex(num_features, metric="angular")

isbn_to_index = {isbn: idx for idx, isbn in enumerate(user_book_matrix.index)}
index_to_isbn = {idx: isbn for isbn, idx in isbn_to_index.items()}

for idx, row in enumerate(sparse_matrix):
    annoy_index.add_item(idx, row.toarray()[0])

annoy_index.build(n_trees=10)

True

- Групиране по ISBN:
    - popular_books.groupby("ISBN"): Групира книгите по ISBN, което означава, че всички оценки за една и съща книга ще бъдат обработени заедно.
- Броене на книгите:
    - .size(): Изчислява броя на записите (оценките) за всяка група. Това ще даде общия брой на оценките за всяка книга в popular_books.
- Сортиране по брой оценки:
    - .sort_values(ascending=False): Сортира групите по брой оценки в низходящ ред, като по този начин най-популярните книги (с най-много оценки) ще бъдат в началото.
- Избиране на 10 най-популярни книги:
    - .index[:10]: Взима индексите (ISBN на книгите) на първите 10 най-популярни книги (с най-много оценки).
    - .tolist(): Преобразува резултата в списък от ISBN на най-популярните книги.

In [43]:
# Fallback to k most popular books
most_popular_books = (
    popular_books.groupby("ISBN")
    .size()
    .sort_values(ascending=False)
    .index[:10]
    .tolist()
)

- Проверка за ISBN:
    - Функцията приема ISBN на книга и броя на съседите (k_neighbors), които да бъдат върнати.
    - if isbn in isbn_to_index:: Проверява дали ISBN на книгата се намира в мапинга на индексите. Ако не, се използват най-популярните книги като резервно решение.
- Извличане на най-близки съседи с Annoy:
    - isbn_idx = isbn_to_index[isbn]: Получава индекса на книгата в Annoy индекса чрез нейното ISBN.
    - annoy_index.get_nns_by_item(isbn_idx, k_neighbors + 1, include_distances=True): Извлича k_neighbors + 1 най-близки съседи на книгата, включително разстоянията. Извлечените резултати съдържат индекси на книги и тяхното разстояние от въпросната книга.
- Подготовка на препоръките:
    - За всеки съседен индекс:
        - neighbor_isbn = index_to_isbn[neighbor_idx]: Преобразува индекса обратно в ISBN.
    - Проверява дали съседната книга е същата като изходната книга и я пропуска, ако е така.
    - Извлича заглавието на съседната книга от combine_book_ratings.
- Връщане на резултати:
    - Ако има намерени препоръки, връща списък с препоръчани книги и тяхното разстояние.
    - Ако ISBN не се намира в индекса, връща списък с най-популярните книги, като резервен вариант.
- Обработка на грешки:
    - Ако възникне грешка, тя се записва и връща като съобщение за грешка.

In [44]:
# Recommendation function
def get_recommends(isbn="", k_neighbors=5):
    try:
        if isbn in isbn_to_index:
            isbn_idx = isbn_to_index[isbn]
            nearest_neighbors = annoy_index.get_nns_by_item(
                isbn_idx, k_neighbors + 1, include_distances=True
            )
            R_books = []
            for neighbor_idx, distance in zip(*nearest_neighbors):
                neighbor_isbn = index_to_isbn[neighbor_idx]
                if neighbor_isbn == isbn:  # Skip the same ISBN
                    continue
                R_book = [
                    combine_book_ratings[combine_book_ratings["ISBN"] == neighbor_isbn][
                        "Book-Title"
                    ].values[0],
                    neighbor_isbn,
                ]
                R_books.append([R_book, distance])
            return [isbn, R_books[::-1]]
        else:
            # Fallback to most popular books
            R_books = [
                [
                    [
                        combine_book_ratings[combine_book_ratings["ISBN"] == book][
                            "Book-Title"
                        ].values[0],
                        book,
                    ],
                    0,
                ]
                for book in most_popular_books[:k_neighbors]
            ]
            return [isbn, R_books]
    except Exception as e:
        pp("ERROR")
        return str(e.with_traceback())

- pp(get_recommends("1558745157")):
    - Извиква функцията get_recommends() с ISBN "1558745157".
    - Функцията ще търси най-близките съседи (книги) в индекса и ще върне списък с препоръчани книги за това ISBN.
    - Резултатът ще бъде отпечатан чрез функцията pp(), която предполагаемо е предназначена за показване на резултата.
- pp(get_recommends("0330281747")):
    - Извиква функцията отново, но с различно ISBN "0330281747".
    - Подобно на предишния случай, ще се търсят препоръки на база на най-близките съседи за това ISBN.

In [45]:
# Test the function
pp(get_recommends("1558745157"))
pp(get_recommends("0330281747"))

['1558745157',
 [[['L Is for Lawless', '0449221490'], 1.3697799444198608],
  [["Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1)",
    '0842329129'],
   1.3697617053985596],
  [['Night Sins', '055356451X'], 1.3676575422286987],
  [['On the Street Where You Live', '0671004530'], 1.3607354164123535],
  [['A Child Called \\It\\": One Child\'s Courage to Survive"', '1558743669'],
   1.2129415273666382]]]
['0330281747',
 [[['Wild Animus', '0971880107'], 0],
  [['The Lovely Bones: A Novel', '0316666343'], 0],
  [['The Da Vinci Code', '0385504209'], 0],
  [['Divine Secrets of the Ya-Ya Sisterhood: A Novel', '0060928336'], 0],
  [['The Red Tent (Bestselling Backlist)', '0312195516'], 0]]]


#### Сравнение между трите вида a-NN имплементации

##### Алгоритми:
1. k-NN
- Предимства: Прост за имплементация и лесно разбираем.
- Недостатъци:
    - Изключително бавен при големи обеми данни.
    - Ниска персонализация, тъй като използва единствено рейтинга на книгите като характеристика.
    - Липсата на допълнителни контекстуални данни (като жанр) ограничава точността на препоръките.
- Извод: Не е подходящ за крайно решение.

2. k-NN с регресия
- Предимства: Въвежда допълнителни характеристики (като възрастта на читателя) за по-персонализирани препоръки.
- Недостатъци:
    - Открита е липса на корелация между възрастта и рейтингите на книгите (p-value = 0), което прави модела логически необоснован.
    - Производителността на модела остава ограничена.
- Извод: Въпреки допълнителния анализ, моделът не показва достатъчно добри резултати и остава неподходящ за практическо приложение.

3. HNSW (Hierarchical Navigable Small World)
- Предимства: Подобрява скоростта и точността на препоръките спрямо стандартния k-NN.
- Недостатъци:
    - Забавяне при работа с големи масиви данни.
    - Ограничен само до популярни книги (топ 10%), което намалява обхвата на препоръките.
- Извод: Макар да е по-добър от стандартния k-NN, HNSW не е достатъчно ефективен за проблеми с големи и разнообразни набори от данни.

4. FAISS (Facebook AI Similarity Search)
- Предимства:
    - Разглежда както популярни, така и непопулярни книги, осигурявайки по-добро покритие.
    - Оптимизира данните чрез TF-IDF, което подобрява производителността и намалява размера на матрицата.
- Недостатъци: Няма сериозни недостатъци при сегашния експеримент.
- Извод: FAISS се оказва най-оптималното и точно решение за текущите данни и цели.

5. Annoy (Approximate Nearest Neighbors)
- Предимства: Лесен за имплементация и добра производителност при умерени набори от данни.
- Недостатъци:
    - По-бавен от FAISS.
    - Лошо се справя с препоръките за непопулярни книги, което ограничава обхвата му.
- Извод: Annoy показва добри резултати, но не превъзхожда FAISS по отношение на производителност и обхват.

##### Общ извод
> Алгоритъмът FAISS е най-подходящото решение за този проблем поради:
>   - Способността си да разглежда както популярни, така и непопулярни книги.
>   - Високата си производителност и оптимизация на данните.
>   - Други алгоритми, като k-NN и неговите модификации, показват по-ниска ефективност или са ограничени от липсата на данни за по-сложна персонализация.

# Конфигурация на проекта

- **Windows / MacOs / Linux**
    - **Стъпка 1**: Направете директория, в която искате да разположите кода ни.
    - **Стъпка 2**: Клонирайте следното repository: https://github.com/Nv4n/soz-project-2024-2025.git на вашата машина в току-що създадената от Вас директория. Клонирането може да стане през:
        - терминала със следната команда: git clone със задаване на директория. Когато клонирате хранилище, можете директно да зададете целева директория, където да бъде изтеглено: git clone <URL-на-репото> <път-към-директорията>
        - UI-я на някое IDE (например Visual Studio Code).
    - **Стъпка 3**: Инсталирайте някой Python Extension Pack и нео=бходимите библиотеки, описани по-нагоре в този документ.
    - **Стъпка 4**: Отворете някой от файловете с изходния код, стартирайте го и експериментирайте.

> **Забележка**: И за трите операционни системи общите стъпки за конфигурация са еднакви. Разликите могат да идват от това какъв терминал използва самата операционна система или как се извикват конкретни команди.